In [ ]:
import numpy as np
import matplotlib.pylab as plt
import sys
import word2vec
import os

In [ ]:
from attalos.imgtxt_algorithms.correlation.correlation import *

In [ ]:
wordvecs_dir = os.environ["WORDVECS_DIR"]
dataset_dir = os.environ["DATASET_DIR"]
octave_eval_dir = os.environ["OCTAVE_EVAL_DIR"]

In [ ]:
w2v_vector_file = os.path.join(wordvecs_dir, "text9Bvin.bin")
w2v_model = word2vec.load(w2v_vector_file)

In [ ]:
len(w2v_model.vocab)

In [ ]:
from oct2py import octave
octave.addpath(octave_eval_dir)

In [ ]:
%matplotlib inline

---

# Load data

In [ ]:
feature_dir = os.path.join(dataset_dir, "alexnet-feats")
data = np.load(os.path.join(feature_dir, "iaprtc_alexfc7.npz"))
with open(os.path.join(feature_dir, "iaprtc_dictionary.txt")) as dictionary:
    D = dictionary.read().splitlines()
with open(os.path.join(feature_dir, "iaprtc_trainlist.txt")) as trainlist:
    train_ims = [ im.split('/')[-1] for im in trainlist.read().splitlines() ]
    train_ims_full = [ im for im in trainlist.read().splitlines() ]
with open(os.path.join(feature_dir, "iaprtc_testlist.txt")) as testlist:
    test_ims_full = [ im for im in testlist.read().splitlines() ]

xTr = data['xTr'].T # xtrain
yTr = data['yTr'].T # ytrain
xTe = data['xTe'].T # xtest
yTe = data['yTe'].T # ytest

In [ ]:
print xTr.shape
print yTr.shape
print xTe.shape
print yTe.shape

---

# Filter out labels not in W2V vocabulary

In [ ]:
to_remove = get_invalid_labels(D, w2v_model.vocab)

In [ ]:
D = filter(lambda word: word not in to_remove.keys(), D)

In [ ]:
yTr = np.delete(yTr, to_remove.values(), axis=1)

In [ ]:
yTe = np.delete(yTe, to_remove.values(), axis=1)

In [ ]:
print yTr.shape
print yTe.shape

In [ ]:
orig_yTr = yTr
orig_yTe = yTe

---

# Construct correlation matrix

In [ ]:
W_L = construct_W(w2v_model, D, dtype=np.float16)

In [ ]:
correlation_arr = np.dot(W_L.T, W_L)

In [ ]:
correlation_arr.shape

In [ ]:
plt.stem(correlation_arr[0])

In [ ]:
correlation_arr_normed = correlation_arr / np.linalg.norm(correlation_arr, 1)

In [ ]:
plt.stem(scale(correlation_arr_normed)[0])

In [ ]:
correlation_arr_normed_nonlinear = nonlinearity(correlation_arr_normed) #np.power(correlation_arr_normed, 3)

In [ ]:
plt.stem(scale(correlation_arr_normed_nonlinear)[0])

In [ ]:
final = scale(correlation_arr_normed_nonlinear)

---

# Transform yTr into correlation space

In [ ]:
yTr = np.dot(orig_yTr, final)

In [ ]:
print xTr.shape
print yTr.shape
print xTe.shape
print yTe.shape

---

# Do linear regression

In [ ]:
# sklearn's Linear Regression implementation
from sklearn import linear_model
n_jobs = -1 # -1 = all CPUs, default = 1
linreg_model = linear_model.LinearRegression(n_jobs=n_jobs)
linreg_model.fit(xTr, yTr)
yHat = linreg_model.predict(xTe)

In [ ]:
# Spark MLLib's Linear Regression implementation
"""
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel
from itertools import izip

def spark_linear_regression(xTr, yTr, xTe):
    for x, y in izip(xTr, yTr):
"""

In [ ]:
# sklearn Polynomial Regression
"""
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
poly_degrees = 2
poly = PolynomialFeatures(degree=poly_degrees)
xTr_poly = poly.fit_transform(xTr)
xTe_poly = poly.fit_transform(xTe)
n_jobs = -1 # -1 = all CPUs, default = 1
linreg_model = linear_model.LinearRegression(n_jobs=n_jobs)
linreg_model.fit(xTr, yTr)
yHat = linreg_model.predict(xTe)
"""

---

# Calculate performance

In [ ]:
[precision, recall, f1score] = octave.evaluate(yTe.T, yHat.T, 5)
print precision
print recall
print f1score

In [ ]:
[precision, recall, f1score] = octave.evaluate(yTe.T, yHat.T, 5)
print precision
print recall
print f1score

---

# Example: Predict labels for an image

In [ ]:
# Randomly select an image
i=np.random.randint(0, yTe.shape[1])

# Run example
imname = os.path.join(dataset_dir, "images", "images", "%s.jpg" % test_ims_full[i])
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
#threshold = np.percentile(yHat[i], 99)
#top_idxs = [idx for idx in yHat[i].argsort()[::-1] if yHat[i][idx] > threshold]
#ypwords = [d_words[idx] for idx in top_idxs]
ypwords = ["%s (%s)" % (D[idx], yHat[i][idx]) for idx in yHat[i].argsort()[::-1][:5]]
#ypwords = [D[j] for j in yHat[i].argsort()[::-1] [ 0:(yHat[i]>0.2).sum() ] ]
# Truth
ytwords = [D[idx] for idx in np.nonzero(yTe[i])[0]]
#ytwords = [D[j] for j in np.where(yTe[i] > 0.5)[0] ]
plt.imshow(im)

print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()

In [ ]:
# Randomly select an image
i=np.random.randint(0, yTe.shape[1])

# Run example
imname = os.path.join(dataset_dir, "images", "images", "%s.jpg" % test_ims_full[i])
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
#threshold = np.percentile(yHat[i], 99)
#top_idxs = [idx for idx in yHat[i].argsort()[::-1] if yHat[i][idx] > threshold]
#ypwords = [d_words[idx] for idx in top_idxs]
ypwords = ["%s (%s)" % (D[idx], yHat[i][idx]) for idx in yHat[i].argsort()[::-1][:5]]
#ypwords = [D[j] for j in yHat[i].argsort()[::-1] [ 0:(yHat[i]>0.2).sum() ] ]
# Truth
ytwords = [D[idx] for idx in np.nonzero(yTe[i])[0]]
#ytwords = [D[j] for j in np.where(yTe[i] > 0.5)[0] ]
plt.imshow(im)

print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()

---

# Expand vocabulary

In [ ]:
def nonlinearity2(arr, percentile=99, power=1):
    for row in arr:
        cutoff = np.percentile(row, percentile)
        for idx, val in enumerate(row):
            if val < cutoff:
                row[idx] = 0
    for idx, row in enumerate(arr):
        arr[idx] = np.power(row, power)
    return arr

In [ ]:
W_V = np.load(os.path.join(wordvecs_dir, "w_v_16.npy")) #construct_W(w2v_model, w2v_model.vocab, dtype=np.float16)
#np.save(os.path.join(wordvecs_dir, "w_v_16_kylez.npy"), W_V)

In [ ]:
#expansion_arr = nonlinearity2(np.dot(W_V.T, W_L), power=2)
#expansion_arr = np.load(os.path.join(wordvecs_dir, "expansion_arr.npy"))
#np.save(os.path.join(wordvecs_dir, "expansion_arr.npy"), expansion_arr)

In [ ]:
expanded = np.dot(yTr, nonlinearity2(np.dot(W_V.T, W_L), power=2).T)
#expanded = np.load(os.path.join(wordvecs_dir, "expanded_yHat.npy"))
#np.save(os.path.join(wordvecs_dir, "expanded_yHat.npy"), expanded)

In [ ]:
expanded.shape

In [ ]:
# Randomly select an image
i = 102 #np.random.randint(0, yTe.shape[1])

# Run example
imname = os.path.join(dataset_dir, "images", "images", "%s.jpg" % test_ims_full[i])
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
#threshold = np.percentile(yHat[i], 99)
#top_idxs = [idx for idx in yHat[i].argsort()[::-1] if yHat[i][idx] > threshold]
#ypwords = [d_words[idx] for idx in top_idxs]
ypwords = [w2v_model.vocab[idx] for idx in expanded[i].argsort()[::-1][:50]]
#ypwords = [D[j] for j in yHat[i].argsort()[::-1] [ 0:(yHat[i]>0.2).sum() ] ]
# Truth
ytwords = [D[idx] for idx in np.nonzero(yTe[i])[0]]
#ytwords = [D[j] for j in np.where(yTe[i] > 0.5)[0] ]
plt.imshow(im)

print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()